In [18]:
from astropy.table import Table
import numpy as np

In [28]:
cat = Table.read('catalogue.fits')

In [34]:
# Run TreeCorr
import treecorr
def runTreecorr(fitscat1,fitscat2,ra_name,dec_name,unitname,e1name,e2name,weightname,
				theta_min = "0.5", theta_max = "300.0", nbins='1000', sep_units = 'arcmin', bin_slop = 1.5):
	cat1 = treecorr.Catalog(fitscat1, ra_col=ra_name, dec_col=dec_name, ra_units=unitname, dec_units=unitname, 
                                          g1_col=e1name, g2_col=e2name, w_col=weightname)
	cat2 = treecorr.Catalog(fitscat2, ra_col=ra_name, dec_col=dec_name, ra_units=unitname, dec_units=unitname, 
                                          g1_col=e1name, g2_col=e2name, w_col=weightname)
	gg = treecorr.GGCorrelation(min_sep=theta_min, max_sep=theta_max, nbins=nbins, sep_units=sep_units, \
            bin_slop=bin_slop)
	gg.process(cat1, cat2)
	return gg

def write_to_file(outfile, col_names, columns, precision):
	ncol = len(col_names)
	data = np.empty( (len(columns[0]), ncol) )
	for i,col in enumerate(columns):
		data[:,i] = col
# 
	width = precision+8
	fmt = '%%%d.%de'%(width, precision)
	# Note: The first one is 1 shorter to allow space for the initial #.
	header = ("#" + "{:^%d}"%(width-1) +
                    " {:^%d}"%(width) * (ncol-1) + "\n").format(*col_names)
	np.savetxt(outfile, data, fmt=fmt, header=header)


In [35]:

ra_name    = 'RAJ2000'
dec_name   = 'DECJ2000'
e1name     = 'gamma1'
e2name     = 'gamma2'
unitname   = 'deg'
sep_units  = 'arcmin'
weightname = None
weightsqname = None
theta_min  = 0.5
theta_max  = 300.0
nbins      = 10

inbinslop = 0.3
num_threads = 4
# nPatches = 50

fitscat1 ='catalogue.fits'
fitscat2 ='catalogue.fits'

gg = runTreecorr(fitscat1,fitscat2,ra_name,dec_name,unitname,e1name,e2name,weightname,
				theta_min = theta_min, theta_max = theta_max, nbins=nbins,sep_units = sep_units,bin_slop = inbinslop)

write_to_file('xipm_treecorr',
                ['r_nom','meanr','meanlogr','xip','xim','xip_im','xim_im','sigma_xip', 'npairs'],
                [ gg.rnom,gg.meanr, gg.meanlogr,gg.xip, gg.xim, gg.xip_im, gg.xim_im, np.sqrt(gg.varxip), gg.npairs], 
				precision=12)
